# Relatorio - 13
#### _Nome_: *Nicolas Enzo Yassuda*

Este relatorio visa obter um maior entendimento do uso do OpenMP, transformando um código sequencial em paralelo, e analisando o desempenho do mesmo.

In [6]:
from IPython.display import display
import pandas as pd

data = {
    'schedule default': [0.000166854, 0.000164176, 0.00016346],
    'schedule static default': [3.00445e-06, 2.82004e-06, 2.87965e-06],
    'schedule static 4parts': [3.0892e-06,3.02587e-06,  2.97744e-06],
    'schedule static 8parts': [2.86568e-06,2.70363e-06,  2.61515e-06],
    'schedule dynamic default': [5.69317e-06,4.65009e-06,  5.23869e-06],
    'schedule dynamic 1': [4.66034e-06,3.86871e-06,  3.64892e-06],
    'schedule dynamic 4': [3.24845e-06,2.96254e-06,  2.42796e-06],
    'schedule dynamic 8': [3.02587e-06,2.69804e-06,  2.77162e-06],
    'schedule guided default': [4.24497e-06,3.86219e-06,  3.81097e-06],
    'schedule guided 2': [3.22703e-06,2.90759e-06,  3.02307e-06],
    'schedule guided 4': [4.81028e-06,2.74647e-06,  2.75578e-06],
    'schedule guided 8': [3.22517e-06,2.87592e-06,  2.69338e-06],
    'schedule auto': [3.11434e-06,2.4382e-06,  2.4084e-06],
    'schedule runtime': [5.00306e-06,4.29526e-06,  4.41447e-06],
    'pi_task': [43.75763, 41.80953, 40.61431],
    'pi_parallel': [1.07930996827781, 1.34939295891672, 1.08233295827322],
    'vetor parallel e critical': [0.00200009,0.00200009,  0.00199986],
    'vetor parallel e pre-alocação': [0.00039994,0.00034232,  0.00039994],
    }

df = pd.DataFrame(data)
display(df)

mean_times = df.mean()

best_time_column = mean_times.idxmin()
best_time_value = mean_times.min()

print(f'O melhor tempo médio foi da coluna: {best_time_column} com o valor: {best_time_value}')

max_variation_column = df.std().idxmax()
max_variation_value = df.std().max()

print(f'A maior variação foi da coluna: {max_variation_column} com o valor: {max_variation_value}')

,schedule default,schedule static default,schedule static 4parts,schedule static 8parts,schedule dynamic default,schedule dynamic 1,schedule dynamic 4,schedule dynamic 8,schedule guided default,schedule guided 2,schedule guided 4,schedule guided 8,schedule auto,schedule runtime
0,0.000167,0.000003,0.000003,0.000003,0.000006,0.000005,0.000003,0.000003,0.000004,0.000003,0.000005,0.000003,0.000003,0.000005
1,0.000164,0.000003,0.000003,0.000003,0.000005,0.000004,0.000003,0.000003,0.000004,0.000003,0.000003,0.000003,0.000002,0.000004
2,0.000163,0.000003,0.000003,0.000003,0.000005,0.000004,0.000002,0.000003,0.000004,0.000003,0.000003,0.000003,0.000002,0.000004


O melhor tempo médio foi da coluna: schedule auto com o valor: 2.6536466666666666e-06
A maior variação foi da coluna: schedule default com o valor: 1.7890209613081612e-06


# Schedules

- Qual scheduler apresentou o menor tempo médio?
  - O scheduler que apresentou o menor tempo médio foi o schedule auto, com um tempo médio de aproximadamente 2,65 microssegundos.
  
- Algum scheduler teve variações significativas entre as execuções? Se sim, por quê?
  - schedule guided 4 apresentou uma variação notável. Seus tempos foram: 4,81 μs, 2,75 μs e 2,76 μs. A diferença entre o tempo máximo e mínimo é de aproximadamente 2,06 μs, o que representa uma variação de cerca de 60% em relação ao tempo médio.
  - schedule auto também mostrou variação significativa. Seus tempos foram: 3,11 μs, 2,44 μs e 2,41 μs, com uma diferença máxima de aproximadamente 0,71 μs, correspondendo a cerca de 26% de variação relativa.
  - A variação ocorre devido à natureza dinâmica desses schedulers e ao overhead associado ao balanceamento de carga em tempo de execução. Schedulers como o dynamic e guided ajustam a distribuição de tarefas entre threads durante a execução, o que pode levar a inconsistências nos tempos medidos em execuções sucessivas, especialmente em tarefas com carga de trabalho muito pequena por iteração.
- Alguma característica específica do trabalho (como carga de dados, balanceamento) parece ter influenciado o comportamento de um scheduler em particular?
  - Sim, a característica específica do trabalho influenciou o comportamento dos schedulers. No código fornecido:

    - O trabalho dentro do loop é mínimo (apenas um push_back por iteração), resultando em carga computacional muito leve por iteração. O número total de iterações é pequeno (n = 64). Essas características fazem com que o overhead de gerenciamento do scheduler tenha um impacto significativo no tempo total de execução. Schedulers que introduzem maior overhead, como o dynamic com pequenos tamanhos de chunk ou o guided, tendem a ter desempenho pior neste cenário porque o tempo gasto na logística de agendamento supera o tempo gasto na execução das tarefas reais. Por outro lado, schedulers como o schedule static com chunks maiores ou o schedule auto permitem reduzir o overhead de agendamento, pois distribuem o trabalho de forma mais estática ou permitem que o compilador otimize a distribuição, melhorando assim o desempenho em tarefas com carga de trabalho leve por iteração.

# Calculo do PI

- Qual abordagem (parallel for ou tasks) apresentou melhor desempenho?

- O valor de MIN_BLK ou o número de tarefas influenciou significativamente o tempo de execução?

- Alguma abordagem teve variação maior entre execuções? Por quê?

#### Qual abordagem (parallel for ou tasks) apresentou melhor desempenho?
A abordagem que apresentou melhor desempenho foi a utilização de parallel for. Analisando os dados fornecidos:

pi_parallel (usando parallel for):

Tempos de execução: 1,07931 s, 1,34939 s, 1,08233 s
Tempo médio: aproximadamente 1,17 segundos
pi_task (usando tarefas):

Tempos de execução: 43,75763 s, 41,80953 s, 40,61431 s
Tempo médio: aproximadamente 42,06 segundos
A abordagem com parallel for foi cerca de 36 vezes mais rápida que a abordagem com tarefas neste cenário.

#### O valor de MIN_BLK ou o número de tarefas influenciou significativamente o tempo de execução?
Valor Elevado de MIN_BLK: MIN_BLK está definido como 268.435.456 (ou seja, 1024 * 1024 * 256). Isso significa que a divisão do trabalho em tarefas só ocorre quando o intervalo de iterações é maior que 256 milhões. Como num_steps é aproximadamente 2 bilhões (2 * 1024^3), o algoritmo cria um número limitado de tarefas (apenas algumas divisões recursivas), resultando em poucas tarefas em relação ao número de threads disponíveis.

Subutilização de Recursos Paralelos: Com poucas tarefas sendo criadas, muitos threads permanecem ociosos ou subutilizados. Isso leva a um baixo aproveitamento do paralelismo, aumentando significativamente o tempo de execução.

Impacto no Tempo de Execução: O alto valor de MIN_BLK impede que o trabalho seja dividido em tarefas menores que poderiam ser executadas em paralelo de forma mais eficiente. Reduzindo o valor de MIN_BLK, mais tarefas seriam criadas, permitindo melhor balanceamento de carga e utilização completa dos recursos disponíveis, o que diminuiria o tempo de execução.

#### Alguma abordagem teve variação maior entre execuções? Por quê?

Sim, a abordagem com parallel for apresentou maior variação entre as execuções em comparação com a abordagem de tarefas. Detalhando os dados:

pi_parallel:

Tempos: 1,07931 s, 1,34939 s, 1,08233 s
Variação absoluta: 0,27008 s
Variação relativa: aproximadamente 23% em relação ao tempo médio
pi_task:

Tempos: 43,75763 s, 41,80953 s, 40,61431 s
Variação absoluta: 3,14332 s
Variação relativa: aproximadamente 7,5% em relação ao tempo médio
Motivos para a Maior Variação na Abordagem parallel for:

Sensibilidade a Cargas de Sistema: Como o tempo de execução é muito menor na abordagem parallel for, ela é mais suscetível a flutuações momentâneas na carga do sistema. Pequenas variações no uso da CPU por outros processos podem ter um impacto proporcionalmente maior.

Overhead de Sincronização e Agendamento: Em execuções mais curtas, o overhead relacionado à criação de threads, sincronização e agendamento tem um peso maior no tempo total. Qualquer atraso nesses processos afeta mais significativamente o desempenho.

Distribuição de Iterações: Dependendo da política de escalonamento (por exemplo, estático vs. dinâmico), pode haver variações na forma como as iterações são distribuídas entre os threads, influenciando o tempo de execução.

Por outro lado, a abordagem com tarefas, tendo um tempo de execução muito maior, é menos afetada por essas flutuações de curto prazo, resultando em uma variação relativa menor entre as execuções.


# Manipulação de Efeitos Colaterais no Vetor
Qual abordagem teve melhor desempenho: omp critical ou pré-alocação de memória?

O uso de omp critical adicionou muito overhead? Como você pode justificar isso?

A ordem dos dados no vetor foi mantida em ambas as abordagens?


A abordagem que teve melhor desempenho foi a pré-alocação de memória.

Análise dos tempos de execução:

Parallel for com omp critical:

Tempos de execução:
0,00200009 segundos
0,00200009 segundos
0,00199986 segundos
Tempo médio: aproximadamente 0,002 segundos

Parallel for com pré-alocação:
Tempos de execução:
0,00039994 segundos
0,00034232 segundos
0,00039994 segundos
Tempo médio: aproximadamente 0,00038 segundos

A abordagem com pré-alocação foi cerca de 5 vezes mais rápida do que a abordagem que utiliza

Sim, o uso de omp critical adicionou um overhead significativo.

Justificativa:

Serialização das Operações:

A diretiva #pragma omp critical garante que a seção de código protegida seja executada por apenas um thread de cada vez.
Isso significa que, embora o loop seja paralelo, a operação dentro da seção crítica (vec.push_back(conta_complexa(i))) é executada sequencialmente.
Como resultado, os threads competem pelo acesso à seção crítica, criando um gargalo que limita o desempenho.
Overhead de Sincronização:

Entrar e sair de uma seção crítica envolve mecanismos de sincronização que consomem tempo.
Em situações onde a operação dentro da seção crítica é rápida (como um simples push_back), o overhead da sincronização pode ser maior do que o tempo gasto na própria operação.
Isso reduz significativamente os benefícios do paralelismo, pois a maior parte do tempo é gasta gerenciando o acesso à seção crítica.
Impacto no Paralelismo:

O objetivo do paralelismo é permitir que múltiplos threads executem operações simultaneamente.
Com a presença de uma seção crítica, os threads são forçados a esperar uns pelos outros, o que diminui a eficiência e aumenta o tempo total de execução.
Na abordagem com pré-alocação de memória, esse overhead é eliminado porque:

O vetor é redimensionado previamente com vec.resize(N), alocando espaço suficiente para todos os elementos.
Cada thread escreve diretamente em uma posição única do vetor (vec[i]), eliminando a necessidade de sincronização.
O acesso ao vetor é feito sem conflitos, permitindo que todos os threads executem suas operações em paralelo sem interferência.

Não, a ordem dos dados no vetor não foi mantida em ambas as abordagens.

Detalhamento:

Abordagem com omp critical e push_back:

O método vec.push_back() adiciona elementos ao final do vetor.
Como múltiplos threads estão adicionando elementos de forma concorrente (apesar da sincronização com omp critical), a ordem em que os elementos são inseridos depende de qual thread chega primeiro à seção crítica.
A ordem de execução dos threads não é garantida e pode variar entre as execuções.
Portanto, os valores em vec podem não estar na ordem crescente de i (de 0 a N-1).
Abordagem com Pré-Alocação e Atribuição Direta:

O vetor é redimensionado para ter exatamente N posições.
Cada thread escreve o resultado de conta_complexa(i) diretamente em vec[i].
Como i varia de 0 a N-1, a ordem dos dados no vetor é mantida conforme esperado.
Não há conflito de acesso, pois cada posição do vetor é única para cada iteração.
Conclusão:

Abordagem com omp critical:
Ordem dos dados não garantida.
Os elementos podem estar fora de ordem devido à natureza não determinística da concorrência entre threads.
Abordagem com Pré-Alocação:
Ordem dos dados mantida.
Cada elemento é colocado na posição correta correspondente ao seu índice i.